## Text mining

Na aula de hoje falaremos sobre modelos de texto e técnicas para trabalhar com strings como emails, transcrições, artigos, SMSs e afins. Estas técnicas também são comumente chamadas de _text mining_.

De uma forma geral, a maior parte do desafio nesse assunto é como transformar o texto em dados numéricos, isto é, fazer um _feature engineering_ que faça sentido para o problema.

Existem diversas maneiras de tratar os problemas de texto em diferentes níveis de profundidade. Este processo de transformar o texto em coordenadas em um espaço vetorial é comumente chamado de _vetorização_ e estabelece relações matemáticas entre o texto, possibilitando realizar operações aritméticas com palavras. Na aula de hoje exploraremos algumas técnicas mais simples e tradicionais para ilustrar tratamentos e aplicações comuns às outras técnicas.

## Natural Language Toolkit

Usaremos bastante a biblioteca NLTK - _Natural Language Toolkit_ (https://www.nltk.org/). Esta biblioteca nos fornece uma miríade de ferramentas úteis para trabalhar com texto, além de _corpora_ de textos em várias línguas.

Vamos utilizar um dataset de texto nesta aula e ir explorando as ferramentas necessárias conforme construímos os modelos. O dataset utilizado será o _Sentiment Labelled Sentences_, que possui comentários/avaliações de sites divididos entre sentimentos positivos e negativos.

In [1]:
import pandas as pd
# Carregando o dataset (IMBD)
df = pd.read_csv('imdb_labelled.txt', sep='	', names=['comment', 'target_sentiment'])

In [2]:
df.head(5)

,comment,target_sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


### Tratamentos

Vamos aplicar alguns tratamentos no texto para construirmos as variáveis. Vamos construir algumas funções para limpá-los:

In [3]:
# Removendo letras maiúsculas
df['comment'] = df['comment'].apply(lambda s: s.lower())

# Removendo pontuação
def remove_punct(s):
    return ''.join([c for c in s if c not in ('.', ',', '-', "'", '"', '!', '?')])
df['comment'] = df['comment'].apply(remove_punct)

Vamos usar a lib `unidecode` para remover outros caracteres especiais:

In [4]:
!pip install unidecode


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Removendo outros caracteres especiais
from unidecode import unidecode
df['comment'] = df['comment'].apply(unidecode)

In [6]:
# Tratando &
df['comment'] = df['comment'].apply(lambda s: s.replace('&', 'and'))

In [7]:
df.head(5)

,comment,target_sentiment
0,a very very very slowmoving aimless movie abou...,0
1,not sure who was more lost the flat character...,0
2,attempting artiness with black and white and c...,0
3,very little music or anything to speak of,0
4,the best scene in the movie was when gerardo i...,1


## Bag of words

Uma das abordagens mais simples e intuitivas para modelar problemas de texto é uma contagem de frequência de palavras. Quando fazemos isto estamos ignorando a _ordem_ das palavras, mas para vários problemas isso é suficiente.

Este módulo do `sklearn` faz automaticamente muitos tratamentos no texto (como a padronização em caracteres minúsculos), mas é importante estar atento a particularidades da língua utilizada, como acentos e flexões específicas.

Vamos construir uma _bag of words_ com nosso dataset:

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
countvec.fit(df['comment'])

CountVectorizer()

In [9]:
words_matrix = countvec.fit_transform(df['comment'])
words_df = pd.DataFrame(words_matrix.todense(), columns=countvec.get_feature_names_out())
df = pd.concat([df, words_df], axis=1)

In [10]:
df.head(3)

,comment,target_sentiment,10,110,12,15,18th,1928,1947,1948,...,youre,yourself,youthful,youtube,youve,yun,zillion,zombie,zombiestudents,zombiez
0,a very very very slowmoving aimless movie abou...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,not sure who was more lost the flat character...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,attempting artiness with black and white and c...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Agora temos um possível problema: existem muitas palavras no dataset, o que acarreta em uma dimensionalidade altíssima!

É conhecido dentro do estudo da linguagem natural que nem toda palavra de uma língua carrega a mesma quantidade de informação - conectivos como "ou" e "e", por exemplo, carregam menos informação do que palavras como "carro", "ruim", "ótimo". Dessa forma, podemos remover do dataset palavras de pouca informação. Estas palavras neste contexto são chamadas de _stopwords_, e a biblioteca NLTK pode nos fornecê-las:

In [11]:
# Carregando stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_english = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danihell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
stopwords_english[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

Temos alguns caracteres especiais nas palavras. Vamos removê-los e tirar duplicidades:

In [13]:
stopwords_english = list(set([s.replace("'", '') for s in stopwords_english]))

In [14]:
stopwords_english[:8]

['through', 'yourself', 'youre', 'them', 'at', 'how', 'havent', 'itself']

In [15]:
# Recarregando o dataset
df = pd.read_csv('imdb_labelled.txt', sep='	', names=['comment', 'target_sentiment'])
# Removendo letras maiúsculas
df['comment'] = df['comment'].apply(lambda s: s.lower())
# Removendo pontuação
df['comment'] = df['comment'].apply(remove_punct)
# Removendo outros caracteres especiais
df['comment'] = df['comment'].apply(unidecode)
# Tratando &
df['comment'] = df['comment'].apply(lambda s: s.replace('&', 'and'))

# Removendo stopwords
def remove_stopwords(s):
    global stopwords_english
    token_list = s.split(' ')
    token_list = [s for s in token_list if s not in stopwords_english]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(remove_stopwords)

In [16]:
df.head(5)

,comment,target_sentiment
0,slowmoving aimless movie distressed drifting y...,0
1,sure lost flat characters audience nearly hal...,0
2,attempting artiness black white clever camera ...,0
3,little music anything speak,0
4,best scene movie gerardo trying find song keep...,1


Ótimo! Conseguimos remover muitas palavras do dataset. Vamos remover também os números:

In [17]:
# Removendo números
def remove_nums(s):
    return ''.join([c for c in s if c not in ('1234567890')])
df['comment'] = df['comment'].apply(remove_nums)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
words_matrix = countvec.fit_transform(df['comment'])
words_df = pd.DataFrame(words_matrix.todense(), columns=countvec.get_feature_names_out())
df = pd.concat([df, words_df], axis=1)

In [19]:
df.head(5)

,comment,target_sentiment,aailiyah,abandoned,ability,about,abroad,absolutely,abstruse,abysmal,...,youdo,young,younger,youthful,youtube,yun,zillion,zombie,zombiestudents,zombiez
0,slowmoving aimless movie distressed drifting y...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,sure lost flat characters audience nearly hal...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,attempting artiness black white clever camera ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,little music anything speak,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,best scene movie gerardo trying find song keep...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Este é um problema _esparso_ - o dataset possui **muitos** zeros. Isto pode representar um problema dependendo do algoritmo utilizado. Vamos ajustar uma Random Forest para termos um baseline de comparação:

In [20]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3, random_state=1312)
model_cols = [c for c in df.columns if (c!='comment' and c!='target_sentiment')]

In [21]:
from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier(n_estimators=10,
                             max_depth=10,
                             class_weight='balanced',
                             min_samples_split=10,
                             n_jobs=-1)

rfr.fit(df_train[model_cols], df_train['target_sentiment'])

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_split=10, n_estimators=10, n_jobs=-1)

In [22]:
from sklearn.metrics import roc_auc_score
print('ROC-AUC treino:', roc_auc_score(df_train['target_sentiment'], rfr.predict_proba(df_train[model_cols])[:,1]))
print('ROC-AUC teste:', roc_auc_score(df_test['target_sentiment'], rfr.predict_proba(df_test[model_cols])[:,1]))

ROC-AUC treino: 0.8479692082111437
ROC-AUC teste: 0.7828354670459934


Conseguimos uma discriminação boa entre as classes, mas podemos melhorá-la considerando uma propriedade simples da linguagem natural: o significado muda pouco em relação a flexões. A versão feminina de uma palavra, por exemplo, carrega praticamente a mesma informação da versão masculina. Podemos aplicar uma "redução" nas palavras para chegar em uma "forma elementar". Este processo é comumente chamado de _stemming_ e faz parte da _tokenização_ do dataset, isto é, separar em palavras (ou n-gramas) relevantes. Vamos aplicar esta técnica:

In [23]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# Exemplo de funcionamento
print(stemmer.stem('comments'))
print(stemmer.stem('practically'))

comment
practic


In [24]:
# Recarregando o dataset
df = pd.read_csv('imdb_labelled.txt', sep='	', names=['comment', 'target_sentiment'])
# Removendo letras maiúsculas
df['comment'] = df['comment'].apply(lambda s: s.lower())
# Removendo pontuação
df['comment'] = df['comment'].apply(remove_punct)
# Removendo outros caracteres especiais
df['comment'] = df['comment'].apply(unidecode)
# Tratando &
df['comment'] = df['comment'].apply(lambda s: s.replace('&', 'and'))

# Removendo stopwords
def remove_stopwords(s):
    global stopwords_english
    token_list = s.split(' ')
    token_list = [s for s in token_list if s not in stopwords_english]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(remove_stopwords)

# Removendo números
def remove_nums(s):
    return ''.join([c for c in s if c not in ('1234567890')])
df['comment'] = df['comment'].apply(remove_nums)

# Stemming
def stemming(s):
    stemmer = PorterStemmer()
    token_list = s.split(' ')
    token_list = [stemmer.stem(s) for s in token_list]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(stemming)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
words_matrix = countvec.fit_transform(df['comment'])
words_df = pd.DataFrame(words_matrix.todense(), columns=countvec.get_feature_names_out())
df = pd.concat([df, words_df], axis=1)
print(df.shape)

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3, random_state=1312)
model_cols = [c for c in df.columns if (c!='comment' and c!='target_sentiment')]

from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier(n_estimators=10,
                             max_depth=10,
                             class_weight='balanced',
                             min_samples_split=10,
                             n_jobs=-1)

rfr.fit(df_train[model_cols], df_train['target_sentiment'])

from sklearn.metrics import roc_auc_score
print('ROC-AUC treino:', roc_auc_score(df_train['target_sentiment'], rfr.predict_proba(df_train[model_cols])[:,1]))
print('ROC-AUC teste:', roc_auc_score(df_test['target_sentiment'], rfr.predict_proba(df_test[model_cols])[:,1]))

(748, 2515)
ROC-AUC treino: 0.883233137829912
ROC-AUC teste: 0.7229334597755652


Já conseguimos uma melhoria no desempenho! Como podemos melhorar ainda mais nossa extração de features?

Um dos jeitos é considerar _n-grams_ do texto, isto é, agrupamentos de n palavras ao invés de somente uma. Vamos reconstruir as variáveis utilizando até 3-grams:

In [26]:
# Recarregando o dataset
df = pd.read_csv('imdb_labelled.txt', sep='	', names=['comment', 'target_sentiment'])
# Removendo letras maiúsculas
df['comment'] = df['comment'].apply(lambda s: s.lower())
# Removendo pontuação
df['comment'] = df['comment'].apply(remove_punct)
# Removendo outros caracteres especiais
df['comment'] = df['comment'].apply(unidecode)
# Tratando &
df['comment'] = df['comment'].apply(lambda s: s.replace('&', 'and'))

# Removendo stopwords
def remove_stopwords(s):
    global stopwords_english
    token_list = s.split(' ')
    token_list = [s for s in token_list if s not in stopwords_english]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(remove_stopwords)

# Removendo números
def remove_nums(s):
    return ''.join([c for c in s if c not in ('1234567890')])
df['comment'] = df['comment'].apply(remove_nums)

# Stemming
def stemming(s):
    stemmer = PorterStemmer()
    token_list = s.split(' ')
    token_list = [stemmer.stem(s) for s in token_list]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(stemming)

# Vamos definir ngram_range no CountVectorizer()
# no mínimo 1 palavra, no máximo 3
countvec = CountVectorizer(ngram_range=(1, 3))
words_matrix = countvec.fit_transform(df['comment'])
words_df = pd.DataFrame(words_matrix.todense(), columns=countvec.get_feature_names_out())
df = pd.concat([df, words_df], axis=1)
print(df.shape)

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3, random_state=1312)
model_cols = [c for c in df.columns if (c!='comment' and c!='target_sentiment')]

from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier(n_estimators=10,
                             max_depth=10,
                             class_weight='balanced',
                             min_samples_split=10,
                             n_jobs=-1)

rfr.fit(df_train[model_cols], df_train['target_sentiment'])

from sklearn.metrics import roc_auc_score
print('ROC-AUC treino:', roc_auc_score(df_train['target_sentiment'], rfr.predict_proba(df_train[model_cols])[:,1]))
print('ROC-AUC teste:', roc_auc_score(df_test['target_sentiment'], rfr.predict_proba(df_test[model_cols])[:,1]))

(748, 14918)
ROC-AUC treino: 0.8130645161290323
ROC-AUC teste: 0.6708550655919077


## TF-IDF

Ao invés de simplesmente contarmos a frequência das palavras, podemos construir uma separação melhor utilizando o método TF-IDF - _Term Frequency-Inverse Document Frequency_. Esta técnica de vetorização consiste basicamente em dividir a frequência da palavra _na amostra_ (contagem) pela frequência da palavra _no dataset_. Dessa forma, conseguimos destacar palavras "raras" porém relevantes, e diminuir a importância de palavras mais corriqueiras.

In [27]:
# Recarregando o dataset
df = pd.read_csv('imdb_labelled.txt', sep='	', names=['comment', 'target_sentiment'])
# Removendo letras maiúsculas
df['comment'] = df['comment'].apply(lambda s: s.lower())
# Removendo pontuação
df['comment'] = df['comment'].apply(remove_punct)
# Removendo outros caracteres especiais
df['comment'] = df['comment'].apply(unidecode)
# Tratando &
df['comment'] = df['comment'].apply(lambda s: s.replace('&', 'and'))

# Removendo stopwords
def remove_stopwords(s):
    global stopwords_english
    token_list = s.split(' ')
    token_list = [s for s in token_list if s not in stopwords_english]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(remove_stopwords)

# Removendo números
def remove_nums(s):
    return ''.join([c for c in s if c not in ('1234567890')])
df['comment'] = df['comment'].apply(remove_nums)

# Stemming
def stemming(s):
    stemmer = PorterStemmer()
    token_list = s.split(' ')
    token_list = [stemmer.stem(s) for s in token_list]
    return ' '.join(token_list)
df['comment'] = df['comment'].apply(stemming)


from sklearn.feature_extraction.text import TfidfVectorizer

# no mínimo 1 palavra, no máximo 3
tfidf = TfidfVectorizer(ngram_range=(1, 3))
words_matrix = tfidf.fit_transform(df['comment'])
words_df = pd.DataFrame(words_matrix.todense(), columns=countvec.get_feature_names_out())
df = pd.concat([df, words_df], axis=1)
print(df.shape)

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3, random_state=1312)
model_cols = [c for c in df.columns if (c!='comment' and c!='target_sentiment')]

from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier(n_estimators=10,
                             max_depth=10,
                             class_weight='balanced',
                             min_samples_split=10,
                             n_jobs=-1)

rfr.fit(df_train[model_cols], df_train['target_sentiment'])

from sklearn.metrics import roc_auc_score
print('ROC-AUC treino:', roc_auc_score(df_train['target_sentiment'], rfr.predict_proba(df_train[model_cols])[:,1]))
print('ROC-AUC teste:', roc_auc_score(df_test['target_sentiment'], rfr.predict_proba(df_test[model_cols])[:,1]))

(748, 14918)
ROC-AUC treino: 0.8633211143695014
ROC-AUC teste: 0.6209894104630946


## Exercícios
- Varie os hiperparâmetros da Random Forest ajustada e tente obter um desempenho melhor dos modelos. Execute múltiplas iterações devido à natureza aleatória dos algoritmos envolvidos.
- Implemente o algoritmo Naïve-Bayes neste dataset e compare seu desempenho com a Random Forest.